In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import zipfile
import re
import dateparser
import collections
from lxml import etree
from xml.etree.ElementTree import XML
from random import shuffle
docxFileName = "quick_section4.docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

In [2]:
mainstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="16"])]/w:t[1]'
first_token_para_xpath = './w:r[1]/w:t[1]'

stations = pd.DataFrame(columns=['station','type','description'])


def is_upper(para):
    if para.xpath(first_token_para_xpath, namespaces=ns):
        if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.isupper():
            return 'station'
        else: 
            return 'new_line'


mainstation = ""
lowerstation = ""
dText = dict()
for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    text = para.xpath('./w:r/w:t', namespaces=ns)
    description = " ".join([t.text for t in text])
    if is_upper(para) == 'station':
        mainxpath = para.xpath(mainstation_xpath, namespaces=ns)
        if mainxpath and len(mainxpath[0].text) > 1:
            mainstation = mainxpath[0].text
    description = description.lstrip('\x01').strip()
    if description:
        if mainstation in dText:
            dText[mainstation].append(description)# += "\n" + description
        else:
            description = re.sub('^(' + re.escape(mainstation) + ')', '\1', description).lstrip('\x01').strip()
            description = re.sub(r" +", " ", description).lstrip('\x01').strip()
            if description:
                dText[mainstation] = [description]

In [3]:
dStations = collections.OrderedDict(dText)

In [4]:
stations = []
descriptions = []
for k in dStations:
    stations.append(k)
    descriptions.append(dStations[k])

In [5]:
stationdf = pd.DataFrame(columns=["Station", "Description"])
stationdf["Station"] = stations
stationdf["Description"] = descriptions

In [6]:
stationdf.head()

,Station,Description
0,ABBEY,[A TOWN [NB] op 3 September 1856** as A; TOWN ...
1,ABBEY & WEST DEREHAM,[[GE] op 1 August 1882 (Thetford & Watton Time...
2,ABBEY FOREGATE,[– see SHREWSBURY.]
3,ABBEY HOUSES,[see BEAUCHIEF .]
4,ABBEY MILLS,[pumping station [private line south of West H...


In [7]:
stationdf_sample = stationdf.sample(n=100, random_state=42)

for i, row in stationdf_sample.iterrows():
    with open("to_annotate/" + str(i) + ".txt", "w") as fw:
        fw.write("[[[STATION: " + row["Station"] + "]]]")
        for d in row["Description"]:
            fw.write("\n" + d)